In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, log_loss, make_scorer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import learning_curve
from sklearn.manifold import TSNE
from umap import UMAP
from scipy.cluster.hierarchy import dendrogram, ward

import optuna

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)

## How to define the different states of the data

In [2]:
#Variable for the imported training data after cleaning up - "train"
#Variable for the feature data set - "train_X"
#Variable for the target series- "train_y"
#Variable for the  target colum - "target"
#Variable for the imported test data set - "test"
#Variable for the transformed test data set - "test_trans"
#Variable for the training data set in the cross validation loop - "X_train"
#Variable for the validation data set in the cross validation loop - "X_val"



In [3]:
train = pd.read_csv(r'../input/playground-series-s3e13/train.csv')
test_1 = pd.read_csv(r'../input/playground-series-s3e13/test.csv')
orig_train = pd.read_csv(r'../input/vector-borne-disease-prediction/trainn.csv')

train.drop('id', axis = 1, inplace = True)
test = test_1.drop('id', axis = 1)

target = 'prognosis'

In [4]:
train = pd.concat([train, orig_train])
print(f'There are {train.duplicated(subset = list(train)[0:-1]).value_counts()[0]} non-duplicate values out of {train.count()[0]} rows in original train dataset')

There are 959 non-duplicate values out of 959 rows in original train dataset


In [5]:
train.shape

(959, 65)

In [6]:
train_X = train.drop(target, axis = 1).copy()
train_y = train[target]

#Fix the issues that the original data set used "-"to separate the words
train_y = [prognosis.replace(' ', '_') for prognosis in train_y]
train_y = np.array(train_y)

In [7]:
train_X.shape, train_y.shape

((959, 64), (959,))

In [8]:
#Cross validation strategy
seed = 42
splits = 3
#cv = RepeatedStratifiedKFold(n_splits = splits, n_repeats = 5, random_state = seed)
cv = StratifiedKFold(n_splits = splits, random_state = seed, shuffle = True)

np.random.seed(seed)

In [9]:
def apk(actual, predicted, k=10):
    
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [10]:
def cross_val_pipe(model, train_X = train_X, train_y = train_y, target ='prognosis', cv = cv, label = ''):
    
    
    #creating encoder and transforming prognosis
    enc = LabelEncoder()
    train_y = enc.fit_transform(train_y)
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train_X), 11)) # Validation predictions are stored in a matrix with length of the number of trainijng samples and # of preds
    train_predictions = np.zeros((len(train_X), 11)) # Same for the train predictions
    train_logloss, val_logloss = [], [] #Store the results from the log_loss calc in a list
    train_map3, val_map3 = [], [] #Store the results from the log_loss calc in a list
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(train_X, train_y)):
                
        model.fit(train_X.iloc[train_idx], train_y[train_idx])
        
        train_preds = model.predict_proba(train_X.iloc[train_idx])
        val_preds = model.predict_proba(train_X.iloc[val_idx])
                  
        train_predictions[train_idx] += train_preds
        val_predictions[val_idx] += val_preds
        
        train_score = log_loss(train_y[train_idx], train_preds)
        val_score = log_loss(train_y[val_idx], val_preds)
        
        train_logloss.append(train_score)
        val_logloss.append(val_score)
        
        #select three most probable prognosis based on train dataset prediction
        train_index = np.argsort(-train_preds)[:,:3] #return index of three most probable prognosis
        
        #select three most probable prognosis based on validation dataset prediction
        val_index = np.argsort(-val_preds)[:,:3]
    
        #calculate map@3
        train_score = mapk(train_y[train_idx].reshape(-1, 1), train_index, 3)
        val_score = mapk(train_y[val_idx].reshape(-1, 1), val_index, 3)
        print(f" The val_score for {fold} is {val_score}")
        
        train_map3.append(train_score)
        val_map3.append(val_score)
    
    print(f'Val log_loss   : {np.mean(val_logloss):.5f} | Train log_loss   : {np.mean(train_logloss):.5f} | {label}')
    print(f'Val MAP@3 Score: {np.mean(val_map3):.5f} | Train MAP@3 Score: {np.mean(train_map3):.5f} | {label}\n')
    
    return val_logloss, val_map3

In [11]:
#Classification models

#List of tuples
models = [
    ('log', LogisticRegression(random_state = seed, max_iter = 10000)),
    #('svc', SVC(random_state = seed, probability = True)),
    #('lda', LinearDiscriminantAnalysis()),
    #('qda', QuadraticDiscriminantAnalysis()),
    #('gauss', GaussianProcessClassifier(random_state = seed)),
    #('et', ExtraTreesClassifier(random_state = seed)),
    #('rf', RandomForestClassifier(random_state = seed)),
    #('xgb', XGBClassifier(random_state = seed, objective = 'multi:softprob', eval_metric = 'map@3')),
    ('lgb', LGBMClassifier(random_state = seed, objective = 'softmax', metric = 'softmax')),
    #('dart', LGBMClassifier(random_state = seed, objective = 'softmax', metric = 'softmax', boosting_type = 'dart')),
    #('cb', CatBoostClassifier(random_state = seed, objective = 'MultiClass', verbose = 0)),
    #('gb', GradientBoostingClassifier(random_state = seed)),
    #('hgb', HistGradientBoostingClassifier(random_state = seed)),
    #('ada', AdaBoostClassifier(random_state = seed)),
    #('knn', KNeighborsClassifier())
]

### Execution

In [12]:
class Decomp:
    def __init__(self, n_components, method="pca", scaler_method='standard'):
        self.n_components = n_components
        self.method = method
        self.scaler_method = scaler_method
        
    def dimension_reduction(self, df):
            
        X_reduced = self.dimension_method(df)
        df_comp = pd.DataFrame(X_reduced, columns=[f'{self.method.upper()}_{_}' for _ in range(self.n_components)], index=df.index)
        
        return df_comp
    
    def dimension_method(self, df):
        X = self.scaler(df)
        if self.method == "pca":
            comp = PCA(n_components=self.n_components, random_state=0)
            X_reduced = comp.fit_transform(X)
        elif self.method == "nmf":
            comp = NMF(n_components=self.n_components, random_state=0)
            X_reduced = comp.fit_transform(X)
        elif self.method == "umap":
            comp = UMAP(n_components=self.n_components, random_state=0)
            X_reduced = comp.fit_transform(X)
        elif self.method == "tsne":
            comp = TSNE(n_components=self.n_components, random_state=0) # Recommend n_components=2
            X_reduced = comp.fit_transform(X)
        else:
            raise ValueError(f"Invalid method name: {method}")
        
        self.comp = comp
        return X_reduced
    
    def scaler(self, df):
        
        _df = df.copy()
            
        if self.scaler_method == "standard":
            return StandardScaler().fit_transform(_df)
        elif self.scaler_method == "minmax":
            return MinMaxScaler().fit_transform(_df)
        elif self.scaler_method == None:
            return _df.values
        else:
            raise ValueError(f"Invalid scaler_method name")
        
    def get_columns(self):
        return [f'{self.method.upper()}_{_}' for _ in range(self.n_components)]
    
    def transform(self, df):
        X = self.scaler(df)
        X_reduced = self.comp.transform(X)
        df_comp = pd.DataFrame(X_reduced, columns=[f'{self.method.upper()}_{_}' for _ in range(self.n_components)], index=df.index)
        
        return df_comp
    @property
        
    def get_explained_variance_ratio(self):
        
        return np.sum(self.comp.explained_variance_ratio_)

In [13]:
def decomp_concat(df):
    global method
    decomp = Decomp(n_components=3, method=method, scaler_method=None)
    df_1 = decomp.dimension_reduction(df).reset_index(drop = True)
    print(f" the shape of df_1 is: {df_1.shape}")
    #df = df.reset_index(inplace = True)
    df = pd.merge(df, df_1, left_index=True, right_index=True)
    print(f" the shape of df_2 2 is: {df.shape}")
    #df = pd.concat([train_X, df], axis=1)
    #df = df.reset_index(inplace = True)
    return df

### Pipeline

In [14]:
def decomp_concat(df):
    global method
    decomp = Decomp(n_components=9, method=method, scaler_method=None)
    df_1 = decomp.dimension_reduction(df).reset_index(drop = True)
    print(f" the shape of df_1 is: {df_1.shape}")
    #df = df.reset_index(inplace = True)
    df = pd.merge(df, df_1, left_index=True, right_index=True)
    print(f" the shape of df_2 2 is: {df.shape}")
    #df = pd.concat([train_X, df], axis=1)
    #df = df.reset_index(inplace = True)
    return df

In [15]:
pca_func_trans = FunctionTransformer(decomp_concat)

In [16]:
pca_func_trans

FunctionTransformer(func=<function decomp_concat at 0x79e6d1d868c0>)

In [17]:
def reset_index(dataframe):
    dataframe = dataframe.reset_index(inplace = False)
    return dataframe

get_reset_index = FunctionTransformer(reset_index, validate=False)

In [18]:
#Set up a logistic regression pipeline
#No feature engineering part )
#pca_pipe = make_pipeline(PCA(n_components=2, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))
pca_pipe = make_pipeline( get_reset_index, pca_func_trans, LogisticRegression(random_state = seed, max_iter = 10000))
umap_pipe = make_pipeline(UMAP(n_components=3, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))

In [19]:
pca_pipe

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function reset_index at 0x79e6d1d30ef0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function decomp_concat at 0x79e6d1d868c0>)),
                ('logisticregression',
                 LogisticRegression(max_iter=10000, random_state=42))])

In [21]:
method = "pca"
cross_val_pipe(pca_pipe)

 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (320, 9)
 the shape of df_2 2 is: (320, 74)
 The val_score for 0 is 0.34010416666666665
 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (320, 9)
 the shape of df_2 2 is: (320, 74)
 The val_score for 1 is 0.3260416666666667
 the shape of df_1 is: (640, 9)
 the shape of df_2 2 is: (640, 74)
 the shape of df_1 is: (640, 9)
 the shape of df_2 2 is: (640, 74)
 the shape of df_1 is: (319, 9)
 the shape of df_2 2 is: (319, 74)
 The val_score for 2 is 0.38610240334378265
Val log_loss   : 2.20143 | Train log_loss   : 1.10697 | 
Val MAP@3 Score: 0.35075 | Train MAP@3 Score: 0.65128 | 



([2.18195494131152, 2.2502745522666814, 2.1720717425052247],
 [0.34010416666666665, 0.3260416666666667, 0.38610240334378265])

In [22]:
def objective(trial):
    param = {
        "loss_function": trial.suggest_categorical("loss_function", ["RMSE", "MAE"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    
    #creating encoder and transforming prognosis
    enc = LabelEncoder()
    train_y = enc.fit_transform(train_y)
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train_X), 11)) # Validation predictions are stored in a matrix with length of the number of trainijng samples and # of preds
    train_predictions = np.zeros((len(train_X), 11)) # Same for the train predictions
    train_logloss, val_logloss = [], [] #Store the results from the log_loss calc in a list
    train_map3, val_map3 = [], [] #Store the results from the log_loss calc in a list
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(train_X, train_y)):
                
        model = CatBoostClassifier(**param)
        model.fit(train_X.iloc[train_idx], train_y[train_idx])
        
        train_preds = model.predict_proba(train_X.iloc[train_idx])
        val_preds = model.predict_proba(train_X.iloc[val_idx])
                  
        train_predictions[train_idx] += train_preds
        val_predictions[val_idx] += val_preds
        
        train_score = log_loss(train_y[train_idx], train_preds)
        val_score = log_loss(train_y[val_idx], val_preds)
        
        train_logloss.append(train_score)
        val_logloss.append(val_score)
        
        #select three most probable prognosis based on train dataset prediction
        train_index = np.argsort(-train_preds)[:,:3] #return index of three most probable prognosis
        
        #select three most probable prognosis based on validation dataset prediction
        val_index = np.argsort(-val_preds)[:,:3]
    
        #calculate map@3
        train_score = mapk(train_y[train_idx].reshape(-1, 1), train_index, 3)
        val_score = mapk(train_y[val_idx].reshape(-1, 1), val_index, 3)
        print(f" The val_score for {fold} is {val_score}")
        
        train_map3.append(train_score)
        val_map3.append(val_score)
    
    print(f'Val log_loss   : {np.mean(val_logloss):.5f} | Train log_loss   : {np.mean(train_logloss):.5f} | {label}')
    print(f'Val MAP@3 Score: {np.mean(val_map3):.5f} | Train MAP@3 Score: {np.mean(train_map3):.5f} | {label}\n')
    
    return val_logloss, val_map3
    

In [23]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=3, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

NameError: name 'optuna' is not defined

In [ ]:
#Create data to plot the learning rate 
for i in range(5,11,1):
    X = train.copy().sample(frac = i/10)
    y = X.pop("prognosis")
    
    #creating encoder and transforming prognosis
    enc = LabelEncoder()
    y = enc.fit_transform(y)
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(X), 11)) # Validation predictions are stored in a matrix with length of the number of trainijng samples and # of preds
    train_predictions = np.zeros((len(X), 11)) # Same for the train predictions
    train_logloss, val_logloss = [], [] #Store the results from the log_loss calc in a list
    train_map3, val_map3 = [], [] #Store the results from the log_loss calc in a list
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
                
        model.fit(X.iloc[train_idx], y[train_idx])
        
        train_preds = model.predict_proba(X.iloc[train_idx])
        val_preds = model.predict_proba(X.iloc[val_idx])
                  
        train_predictions[train_idx] += train_preds
        val_predictions[val_idx] += val_preds
        
        train_score = log_loss(y[train_idx], train_preds)
        val_score = log_loss(y[val_idx], val_preds)
        
        train_logloss.append(train_score)
        val_logloss.append(val_score)
        
        #select three most probable prognosis based on train dataset prediction
        train_index = np.argsort(-train_preds)[:,:3] #return index of three most probable prognosis
        
        #select three most probable prognosis based on validation dataset prediction
        val_index = np.argsort(-val_preds)[:,:3]
    
        #calculate map@3
        train_score = mapk(y[train_idx].reshape(-1, 1), train_index, 3)
        val_score = mapk(y[val_idx].reshape(-1, 1), val_index, 3)
        
        train_map3.append(train_score)
        val_map3.append(val_score)
    
    print(f'Val log_loss  for {i} : {np.mean(val_logloss):.5f} | Train log_loss   : {np.mean(train_logloss):.5f}')
    print(f'Val MAP@3 Score for {i}: {np.mean(val_map3):.5f} | Train MAP@3 Score: {np.mean(train_map3):.5f}' )

# Create pipeline where the best estimator type can selected

In [ ]:
pca_pipe = make_pipeline(PCA(n_components=2, random_state=0), model)
#pca_pipe = make_pipeline(PCA(n_components=2, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))
#pca_pipe = make_pipeline( get_reset_index, pca_func_trans, LogisticRegression(random_state = seed, max_iter = 10000))
#umap_pipe = make_pipeline(UMAP(n_components=3, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))

In [ ]:
logloss_list, map3_list = pd.DataFrame(), pd.DataFrame()

for (label, model) in models: #Aha, the label in the print function comes from here, as the models had been setuo as a list of tuples
    model_pipeline = make_pipeline(PCA(n_components=2, random_state=0), model)
    (logloss_list[label], map3_list[label]) = cross_val_pipe(model_pipeline, label = label)

In [ ]:
logloss_list_v1, map4_list = pd.DataFrame(), pd.DataFrame()

for (label, model) in transformers: #Aha, the label in the print function comes from here, as the models had been setuo as a list of tuples
    (logloss_list_v1[label], map4_list[label]) = cross_val_pipe(model, label = label)